# NYC_cabs

In [1]:
#Importation des Modules nécessaires
import pandas as pd


#Pour que python reconnaisse un dossier comme un module il faut ajouter en son sein un fichier __init__.py 
#et également l'ajouter au PATH

import sys
sys.path.insert(0, "/home/apprenant/PycharmProjects/NYC_cabs")


#On peut ensuite importer le dossier comme un module

from src.utils import *


In [2]:
df = pd.read_csv("/home/apprenant/PycharmProjects/NYC_cabs/data/clean.csv",parse_dates= [2,3])


In [3]:

df.dtypes


id                           object
vendor_id                     int64
pickup_datetime      datetime64[ns]
dropoff_datetime     datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
trip_duration                 int64
dtype: object

## 1. Exploration des données

## 1.1 calculer la vitesse moyenne de chaque trajet (en km/h) 

In [4]:
# pour ce calcul il faut la distance en km, le temps de trajet en heure.


# cooordonnées GPS en radians du 1er point (ici, mairie de Tours)
latA = deg2rad(47.390668)  # Nord
longA = deg2rad(0.689319)  # Est

# cooordonnées GPS en radians du 2ème point (ici, mairie de Limoges)
latB = deg2rad(45.826516)  # Nord
longB = deg2rad(1.260290)  # Est

dist_km = (distanceGPS(latA, longA, latB, longB))/1000
print(int(dist_km),"km")

179 km


In [8]:
df['distance'] = ft_haversine_distance(df['pickup_latitude'].values,df['pickup_longitude'].values, df['dropoff_latitude'].values,df['dropoff_longitude'].values)


In [9]:
df["trip_duration"] = df["trip_duration"]/3600
df["speed"] = df["distance"] / df["trip_duration"]

In [10]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,distance,speed
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,-73.982155,40.767937,-73.964630,40.765602,0.000035,1.498521,42683.141328
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,-73.980415,40.738564,-73.999481,40.731152,0.000051,1.805507,35293.171806
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,-73.979027,40.763939,-74.005333,40.710087,0.000164,6.385098,38959.923022
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,-74.010040,40.719971,-74.012268,40.706718,0.000033,1.485498,44876.595709
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,-73.973053,40.793209,-73.972923,40.782520,0.000034,1.188588,35411.738926


## 1.2 calculer le nombre de trajets effectués en fonction du jour de la semaine 

In [15]:

df['weekday'] = df['pickup_datetime'].dt.day_name()
print(df.groupby(df['weekday']).count()['id'])

weekday
Friday       223533
Monday       187418
Saturday     220868
Sunday       195366
Thursday     218574
Tuesday      202749
Wednesday    210136
Name: id, dtype: int64


## 1.3 calculer le nombre de trajets effectués en fonction de l’horaire de la journée par tranche de 4h.

In [18]:
df['hour'] = df['pickup_datetime'].dt.hour
print(df.groupby(df['hour']).count()['id'])

hour
0     53248
1     38571
2     27972
3     20895
4     15792
5     15002
6     33248
7     55600
8     67053
9     67663
10    65437
11    68476
12    71873
13    71473
14    74292
15    71811
16    64313
17    76483
18    90600
19    90308
20    84072
21    84185
22    80492
23    69785
Name: id, dtype: int64


In [37]:
# a = df.loc[(df['hour'] >= 0) & (df['hour'] <= 3)]
# print("de 0h à 3h",a.count()['id'],"trajets effectués")

# b = df.loc[(df['hour'] >= 4) & (df['hour'] <= 7)]
# print("de 4h à 7h",b.count()['id'],"trajets effectués")

# c= df.loc[(df['hour'] >= 8) & (df['hour'] <= 11)]
# print("de 8h à 11h",c.count()['id'],"trajets effectués")

# d = df.loc[(df['hour'] >= 12) & (df['hour'] <= 15)]
# print("de 12h à 15h",d.count()['id'],"trajets effectués")

# e = df.loc[(df['hour'] >= 16) & (df['hour'] <= 19)]
# print("de 16h à 19h",e.count()['id'],"trajets effectués")

# f = df.loc[(df['hour'] >= 20) & (df['hour'] <= 23)]
# print("de 20h à 23h",f.count()['id'],"trajets effectués")

list_0_to_4 = []
list_5_to_8 = []
list_9_to_12 = []
list_13_to_16 = []
list_17_to_20 = []
list_21_to_23 = []
for value in df['hour'] : 
    if value in [0 ,1, 2 ,3 ,4] : 
        list_0_to_4.append(value)
    elif value in [5,6,7,8] : 
        list_5_to_8.append(value)
    elif value in [9,10,11,12] :
        list_9_to_12.append(value)
    elif value in [13,14,15,16] :
        list_13_to_16.append(value)
    elif value in [17,18,19,20] : 
        list_17_to_20.append(value)
    else : 
        list_21_to_23.append(value)
        
print(len(list_0_to_4))
print(len(list_5_to_8))
print(len(list_9_to_12))
print(len(list_13_to_16))
print(len(list_17_to_20))
print(len(list_21_to_23))

156478
170903
273449
281889
341463
234462


## 1.4 calculer le nombre de km parcourus par jour de la semaine

In [38]:

df_km_day = df.groupby(df['weekday']).distance.sum()
print(round(df_km_day,2))

weekday
Friday       758724.47
Monday       668482.11
Saturday     736411.13
Sunday       726453.25
Thursday     747677.63
Tuesday      678328.10
Wednesday    702918.80
Name: distance, dtype: float64


## 2. Choix du modèle de machine learning

## 3. Préparation des données

### 3.a échantillonage

### 3.b Sélection de la variable cible et des variables à explicatives

## 4. Division du jeu de données en training/validation/testing sets

## 5. Apprentissage sur le training set

#### On implémente la regression linéaire

#### On obtient ainsi les parmètres β1 et β0 de notre modèle

#### On peut à présent réaliser des prédictions

#### enfin on peut représenter notre modèle linéaire

## 6. Evaluation du modèle sur le testing set

#### évaluation à partir du R2

#### évaluation à partir du RMSE

#### évaluation graphique